In [71]:
# toutes les importations nécessaires
#%pip install ortools # -> si nécessaire, pour installer le module ortools, décommenter cette ligne
import random
import math
from tqdm import tqdm # type: ignore
import matplotlib.pyplot as plt
from ortools.sat.python import cp_model # type: ignore
import time
import numpy as np

In [72]:
def lire_fichier(fichier): # Fonction pour lire le fichier d'entrée
    with open(fichier, 'r') as f:
        liste_items = []
        data = False
        max_capacity = None
        for ligne in f:
            ligne = ligne.strip()
            if ligne.startswith('MAX_CAPACITY:'):
                max_capacity = int(ligne.split()[1])
            elif ligne.startswith('DATA [id profit weight]:'):
                data = True
                continue
            if data and ligne:
                liste_items.append([int(x) for x in ligne.split()])
    return max_capacity, liste_items

In [73]:
# résolution linéaire de knapsack
def solve_knapsack_cp_sat(max_capacity, liste_items):
    #max_capacity, liste_items = lire_fichier(fichier)
    n = len(liste_items)
    model = cp_model.CpModel()
    
    x = []
    for i in range(n): # on crée une variable binaire pour chaque item : x_i = 1 si l'item i est pris, 0 sinon
        x.append(model.NewBoolVar(f"x_{i}"))

    model.Add(
        sum(liste_items[i][2] * x[i] for i in range(n)) <= max_capacity # fonction contrainte : somme des poids des items sélectionnés <= max_capacity
    )

    model.Maximize(
        sum(liste_items[i][1] * x[i] for i in range(n)) # fonction objectif : somme des profits des items sélectionnés
    )

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return solver.ObjectiveValue()
        # poids_total = 0
        # items_selectionnes = []
        # for i in range(n):
        #     if solver.Value(x[i]) == 1:
        #         idx_item = liste_items[i][0]
        #         weight_item = liste_items[i][2]
        #         items_selectionnes.append(idx_item)
        #         poids_total += weight_item

        #print(f"Poids total = {poids_total}")
        #print(f"Items sélectionnés (IDs) = {items_selectionnes}")
    else:
        print("Aucune solution trouvée.")
        return None

In [ ]:
# Recuit simulé

def calculer_poids_total(liste_items, solution): 
    return sum(liste_items[i][2] for i in range(len(liste_items)) if solution[i] == 1)

def calculer_profit_total(liste_items, solution):
    return (-sum(liste_items[i][1] for i in range(len(liste_items)) if solution[i] == 1))

def verifier_solution(liste_items, solution, max_capacity):
    return calculer_poids_total(liste_items, solution) <= max_capacity

def generer_solution_aleatoire(liste_items):
    return [random.randint(0, 1) for _ in range(len(liste_items))]

def generer_voisin(solution):
    index = random.randint(0, len(solution) - 1)
    solution[index] = 1 if solution[index] == 0 else 0
    return solution

def accepter_solution_moins_bonne(delta, temperature):
    if temperature == 0:
        return 1.0
    else:
        proba = math.exp(-delta / temperature)
        return random.random() < proba

def sort_items(liste_items):
    return sorted(liste_items, key=lambda x: x[1] / x[2], reverse=True)

def f_solution_initiale(liste_items, capacite_max):
    solution = [0] * len(liste_items)
    poids_total = 0
    for i in range(len(liste_items)):
        if poids_total + liste_items[i][2] <= capacite_max:
            solution[i] = 1
            poids_total += liste_items[i][2]
        else:
            break
    return solution

def recuit_simule(liste_items, capacite_max, temperature_initiale, n1, n2, mu, solution_initiale):
    solution = solution_initiale[:]
    meilleur_solution = solution_initiale[:]
    for i in range(n1):
        temperature = temperature_initiale * (mu ** i)
        for j in range(n2):
            voisin = generer_voisin(solution[:])
            if not verifier_solution(liste_items, voisin, capacite_max):
                continue
            delta = calculer_profit_total(liste_items, voisin) - calculer_profit_total(liste_items, solution)
            if delta < 0 or accepter_solution_moins_bonne(delta, temperature):
                solution = voisin
                if calculer_profit_total(liste_items, solution) < calculer_profit_total(liste_items, meilleur_solution):
                    meilleur_solution = solution[:]
    return meilleur_solution

In [75]:
# fonction de test et de stockage des résultats pour le recuit simulé   
def tester_recuit_simule(liste_items, capacite_max, solution_initiale, temperatures_initiales, n1, n2, mus, num_runs):
    results = {}
    total_iterations = len(n1) * len(n2) * len(temperatures_initiales) * len(mus) * num_runs
    with tqdm(total=total_iterations, desc="Test des paramètres") as pbar_config:
        for n1_val in n1:
            for n2_val in n2: 
                for temperature_initiale in temperatures_initiales:
                    for mu in mus:              
                        profits = []
                        for _ in range(num_runs):
                            solution = recuit_simule(liste_items, capacite_max, temperature_initiale, n1_val, n2_val, mu, solution_initiale)
                            profit = -calculer_profit_total(liste_items, solution)
                            profits.append(profit)
                            pbar_config.update(1)
                        results[(n1_val, n2_val, temperature_initiale, mu)] = profits
    return results

In [76]:
def afficher_graphes_parametres(results, n1, n2, temperatures_initiales, mus):
    profits_par_temp = {}
    profits_par_n1 = {}
    profits_par_n2 = {}
    profits_par_mu = {}

    for (n1_val, n2_val, temp, mu), profits in results.items():
        mean_profit = sum(profits) / len(profits)

        profits_par_n1.setdefault(n1_val, []).append(mean_profit)
        profits_par_n2.setdefault(n2_val, []).append(mean_profit)
        profits_par_temp.setdefault(temp, []).append(mean_profit)
        profits_par_mu.setdefault(mu, []).append(mean_profit)

    mean_profits_n1 = {n1_val: sum(profits) / len(profits) for n1_val, profits in profits_par_n1.items()}
    mean_profits_n2 = {n2_val: sum(profits) / len(profits) for n2_val, profits in profits_par_n2.items()}
    mean_profits_temp = {temp: sum(profits) / len(profits) for temp, profits in profits_par_temp.items()}
    mean_profits_mu = {mu: sum(profits) / len(profits) for mu, profits in profits_par_mu.items()}

    median_n1 = np.median(n1)
    median_n2 = np.median(n2)
    median_temp = np.median(temperatures_initiales)
    median_mu = np.median(mus)


    plt.figure()
    plt.plot(list(mean_profits_temp.keys()), list(mean_profits_temp.values()), marker='o')
    plt.title(f'Profit moyen en fonction de temperature_initiale, (n1={median_n1}, n2={median_n2}, mu={median_mu})')
    plt.xlabel('temperature_initiale')
    plt.ylabel('Profit moyen')
    plt.grid(True)
    plt.show()
    
    plt.figure()
    plt.plot(list(mean_profits_n1.keys()), list(mean_profits_n1.values()), marker='o')
    plt.title(f'Profit moyen en fonction de n1 (max_iter), (n2={median_n2}, temp={median_temp}, mu={median_mu})')
    plt.xlabel('n1 (max_iter)')
    plt.ylabel('Profit moyen')
    plt.grid(True)
    plt.show()
    
    plt.figure()
    plt.plot(list(mean_profits_n2.keys()), list(mean_profits_n2.values()), marker='o')
    plt.title(f'Profit moyen en fonction de n2, (n1={median_n1}, temp={median_temp}, mu={median_mu})')
    plt.xlabel('n2')
    plt.ylabel('Profit moyen')
    plt.grid(True)
    plt.show()
    
    plt.figure()
    plt.plot(list(mean_profits_mu.keys()), list(mean_profits_mu.values()), marker='o')
    plt.title(f'Profit moyen en fonction de mu, (n1={median_n1}, n2={median_n2}, temp={median_temp})')
    plt.xlabel('mu')
    plt.ylabel('Profit moyen')
    plt.grid(True)
    plt.show()

In [77]:
fichiers = [
        'pi-12-100-1000-001.kna',
        #'pi-12-1000-1000-001.kna',
        #'pi-12-10000-1000-001.kna',
        'pi-13-100-1000-001.kna',
        #'pi-13-1000-1000-001.kna',
        #'pi-13-10000-1000-001.kna',
        'pi-15-100-1000-001.kna',
        #'pi-15-1000-1000-001.kna',
        #'pi-15-10000-1000-001.kna'
    ]

# Paramètres pour le recuit simulé
temperatures_initiales = [500, 5000, 10000]
n1 = [10, 50, 100, 250, 500, 750, 1000]
n2 = [10, 100, 1000]
mus = [0.01, 0.1, 0.25]


# Exemple d'utilisation
# fichier = 'pi-12-100-1000-001.kna'
# capacite_max, liste_items = lire_fichier(fichier)
# print("Capacité max = ", capacite_max)
# solution_initiale = f_solution_initiale(liste_items, capacite_max)
# print("Solution initiale : ", solution_initiale)
# temperature_initiale = 1000
# n1 = 100
# n2 = 1000
# mu = 0.1
# print("Paramètres : n1 = ", n1, ", n2 = ", n2, ", mu = ", mu)
# result = recuit_simule(liste_items, capacite_max, temperature_initiale, n1, n2, mu, solution_initiale)
# print("Profit total : ", -calculer_profit_total(liste_items, result))

temperatures_initiales = [10, 100, 1000]
n1 = [10, 100, 1000]
n2 = [10, 100, 1000]
mus = [0.01, 0.1, 0.25]

for fichier in fichiers:
    print(f"\n--- Résultats pour {fichier} ---")
    print("-" * 20)
    capacite_max, liste_items = lire_fichier(fichier)
    #print("Capacité max = ", capacite_max)
    start_time = time.time()
    profit_max = solve_knapsack_cp_sat(capacite_max, liste_items)
    end_time = time.time()
    print(f"Temps de résolution CP-SAT: {end_time - start_time:.2f} secondes")
    print("Profit max : ", profit_max)
    print("-" * 20)
    
    start_time = time.time()
    liste_items = sort_items(liste_items)
    solution_initiale = f_solution_initiale(liste_items, capacite_max)
    #print("Solution initiale : ", solution_initiale)
    results = tester_recuit_simule(liste_items, capacite_max, solution_initiale, temperatures_initiales, n1, n2, mus, 10)
    end_time = time.time()
    print(f"Temps de résolution Recuit Simulé: {end_time - start_time:.2f} secondes")
    print("-" * 20)
    afficher_graphes_parametres(results, temperatures_initiales, n1, n2, mus)
    
    profits_moy = []
    for params, profits in results.items():
        profit_moy = sum(profits) / len(profits)
        profits_moy.append((profit_moy, params, profits))
    
    profits_moy.sort(key=lambda x: x[0], reverse=True)
    profits_moy = profits_moy[:5]  # Top 5 configurations
    print("\n--- Top 5 configurations ---")
    for profit_moy, params, profits in profits_moy:
        print(f"Parameters: max_iter={params[0]}, temp_init={params[1]}, alpha={params[2]}")
        print(f"  Moyenne : {profit_moy}")
        print(f"  Profits : {profits}")
        print(f"  Ecart-type: {math.sqrt(sum([(x - profit_moy)**2 for x in profits]) / len(profits))}")
        print("-" * 20)


--- Résultats pour pi-12-100-1000-001.kna ---
--------------------
Temps de résolution CP-SAT: 0.02 secondes
Profit max :  970.0
--------------------


Test des paramètres:  67%|██████▋   | 540/810 [01:34<00:47,  5.71it/s] 


ZeroDivisionError: float division by zero